# **Problem Set 1**

> *Instructions:*  Save a copy of this Colab notebook to your own Google Drive folder **adding your name to the filename** and open in Google Colaboratory. When you have completed the exercise you will need to upload a .pdf of the Colab notebook to canvas. Instructions on how to download a .pdf of this notebook are found at the end of the exercise.

> *Documentation:* Additional resources on how to use Google Colab, enter answers, view code, and which Python libraries are used can be found in the documentation file.

>    *Honor Statement:* Recall that your name on this worksheet is considered to be a reaffirmation of your commitment to the Dartmouth Honor Principle. No online resources, phones, or calculators are allowed on these worksheets, although you may use your textbooks and notes.

---



##Don't forget that you need to run every code block **including** the ones where you input information. This does not include *text blocks* where you are writing your answers.

---

###*Please fill in your name below by double clicking the text*

Type your name here

---

##**Below you will be interacting with and running code to answer questions about the observability of objects from several observatories from around the United States. Read the directions carefully and consult the worksheet documentation for additional instruction and resources.**

##Begin by running the block of code below using the play button on the left side to import all relevant packages and define functions for use later. *You can ignore the warning about this notebook not being authored in Google.*

In [ ]:
#@title  { vertical-output: true, display-mode: "form" }

##this block imports all the libraries and functions used in the exercise
!pip install astropy
!pip install astroplan
import astroplan
import astropy.units as u
from astroplan import Observer
from astropy.time import Time
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib
from datetime import datetime, timedelta
import warnings
from astropy.coordinates import solar_system_ephemeris, EarthLocation
from astropy.coordinates import get_body_barycentric, get_body, get_moon
from astropy.visualization import astropy_mpl_style, quantity_support
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from astropy.coordinates import get_sun
from astropy.coordinates import get_moon
import matplotlib.pyplot as plt
plt.style.use(astropy_mpl_style)
quantity_support()
from pytz import timezone
from astroplan import Observer
from astroplan import FixedTarget
from astroplan.plots import plot_sky,plot_sky_24hr,plot_altitude
from google.colab import widgets
import matplotlib.pyplot as plt
from astropy.time import Time
from astroplan.plots import plot_airmass
from astroplan import Observer
from astroplan import FixedTarget
from astropy.coordinates import Angle

### The blocks below defines all the functions used in the exercise

#this function converts 24 hr UTC time back to a given local time zone
def convert_utc_time(moon_sun_times,your_tz):
  converted_times_list = []
  for i in range(len(moon_sun_times)):
    inDate = str(moon_sun_times[i])
    new_date = dt.datetime.strptime(inDate, '%Y-%m-%d %H:%M:%S.%f')
    converted = new_date + timedelta(hours=your_tz)
    converted_times_list.append(converted)
  return(converted_times_list)

#this function takes the location information for the observatory and the date and time
#and computes the moon/sun rise and set times, twilight start/end times, and solar midnight time
def get_rise_set_times(observatory,obs_date_time):
  obs_location = Observer(longitude=observatory[2]*u.deg, latitude=observatory[1]*u.deg,
                  elevation=observatory[3]*u.m, name=observatory[0], timezone=observatory[4])
  time = Time(obs_date_time)
  moon_rise = obs_location.moon_rise_time(time) 
  moon_rise.format = 'iso'
  moon_set = obs_location.moon_set_time(time) 
  moon_set.format = 'iso'
  sun_rise = obs_location.sun_rise_time(time) 
  sun_rise.format = 'iso'
  sun_set = obs_location.sun_set_time(time) 
  sun_set.format = 'iso'
  twilight_begin = obs_location.twilight_evening_astronomical(time) 
  twilight_begin.format = 'iso'
  twilight_end = obs_location.twilight_morning_astronomical(time)
  twilight_end.format = 'iso'
  solar_midnight = obs_location.midnight(time)
  solar_midnight.format = 'iso'
  moon_sun_times = [moon_rise,moon_set,sun_rise,sun_set,twilight_begin,twilight_end,solar_midnight]
  return(obs_location,moon_sun_times)

#this is a function that retrieves the coordinate information for a given celestial object
#creating a SkyCoord object by querying for a name 
def get_object_coord(object_name):
    celestial_object = SkyCoord.from_name(object_name)
    return(celestial_object)

#this function takes the location and object information and plots the airmass of the object
#over a 24-hr period of time
def plot_object_airmass(object_name,obs_location,obs_date_time,obs_choice):
  target = FixedTarget.from_name(object_name)
  your_time = Time(obs_date_time)
  transit_time = obs_location.target_meridian_transit_time(your_time, target,
                                                       which="next") 
  transit_time.format = 'iso'
  fig, axes = plt.subplots(figsize=(20,10))
  plt.yticks(fontsize=20)
  plt.xticks(fontsize=20)
  plt.xlabel('Time',fontsize=25)
  plt.ylabel('Altitude',fontsize=25)
  your_tz = observatory[5]
  solar_midnight = converted_dates[6]
  observe_time = Time(solar_midnight)
  object_coord = SkyCoord.from_name(object_name,frame='icrs')  
  altair1 = FixedTarget(name=object_name, coord=object_coord)
  style1 = {'linewidth': 5}
  plot_airmass(altair1, obs_location, transit_time, brightness_shading=True,altitude_yaxis=True,style_kwargs=style1,use_local_tz=False)
  plt.yticks(fontsize=20)
  plt.ylabel('Altitude [degrees]',fontsize=25)
  plt.title(object_name+' as observed from '+obs_choice,fontsize=35)
  return(plt.show(),transit_time)

#this function converts RA or DEC in degrees to hours,min,sec and was retrieved from
#http://www.bdnyc.org/2012/10/decimal-deg-to-hms/
def deg2HMS(ra='', dec='', round=False):
  RA, DEC, rs, ds = '', '', '', ''
  if dec:
    if str(dec)[0] == '-':
      ds, dec = '-', abs(dec)
    deg = int(dec)
    decM = abs(int((dec-deg)*60))
    if round:
      decS = int((abs((dec-deg)*60)-decM)*60)
    else:
      decS = (abs((dec-deg)*60)-decM)*60
    DEC = '{0}{1} {2} {3}'.format(ds, deg, decM, decS)
  if ra:
    if str(ra)[0] == '-':
      rs, ra = '-', abs(ra)
    raH = int(ra/15)
    raM = int(((ra/15)-raH)*60)
    if round:
      raS = int(((((ra/15)-raH)*60)-raM)*60)
    else:
      raS = ((((ra/15)-raH)*60)-raM)*60
    RA = '{0}{1} {2} {3}'.format(rs, raH, raM, raS)
  if ra and dec:
    return (RA, DEC)
  else:
    return RA or DEC

##<u>*Step 1*</u> **To start this problem, run the block of code below to set the Date in the block below to April 1st, 2020.** 
##In future steps you will be asked to change the Date and Time for which the calculations apply, which can be done by simply changing the numbers in the obs_date and obs_time forms and re-running the block of code. All the code uses a 24 hour clock (in hours, minutes, and seconds), so for example, 6pm is 18:00:00. 

In [ ]:
#@title Input the date and time for the calculations by clicking the arrows on the date chooser. Input time by typing it into the form. **Note:** time MUST be in HH:MM:SS 24-hr format
obs_date =  "2020-04-01"#@param {type:"date"}
obs_time = "09:00:00" #@param {type:"string"}
obs_date_time = obs_date +' '+obs_time
print('You successuflly set your observation date and time to: ', obs_date_time)


##<u>*Step 2*</u> Now select `Shattuck Observatory, Dartmouth' and run the block of code

In [ ]:
#@title Choose the observatory from the dropdown that you'd like to use
obs_choice = "Mauna Kea Observatory, Hawaii" #@param ["Shattuck Observatory (Dartmouth)", "Mauna Kea Observatory, Hawaii", "Kitt Peak Observatory, Arizona"]

#depending on choice define variable that stores observatory information
if obs_choice == "Shattuck Observatory (Dartmouth)":
  obs_lat =43.705000
  obs_long =-72.285278
  obs_elev = 171
  obs_tz = "US/Eastern"
  obs_utc = -5
if obs_choice == "Mauna Kea Observatory, Hawaii":
  obs_lat =19.824035111845326
  obs_long =-155.4689621767943
  obs_elev =4205
  obs_tz = "US/Hawaii"
  obs_utc = -10
if obs_choice == "Kitt Peak Observatory, Arizona":
  obs_lat =31.951841
  obs_long =-111.615948
  obs_elev = 1923
  obs_tz = "US/Mountain"
  obs_utc = -7

#store observatory information as variables in a list
observatory = [obs_choice,obs_lat,obs_long,obs_elev,obs_tz,obs_utc]
print("You successfully set your observatory choice to ", obs_choice)

#use observatory info and date/time to create location object
#and calculate moon/sun rise/set times, solar midnight, and twilight times
obs_location, moon_sun_times = get_rise_set_times(observatory,obs_date_time)

#convert time from UTC to local time zone
converted_dates = convert_utc_time(moon_sun_times,observatory[5])
new_list = []
for i in range(len(converted_dates)):
  times = converted_dates[i].strftime("%Y-%m-%d, %H:%M:%S.%f")
  times = times[:-7]
  new_list.append(times)

##<u>*Step 3*</u> Running the block of code below first shows you a 2D representation of the sky over a 24-hr period (Sky Plot). Planets are shown with unique colors/symbols for the specified location and time. The plot on the right is of the position of the planets in the sky over time with color changes representing a change from daylight (white) to full darkness (dark grey). 

In [ ]:
#@title  { vertical-output: true, display-mode: "form" }

t = Time(obs_date + ' 00:00:00')
t2 = Time(obs_date_time)

#create location object from observatory information
loc = EarthLocation(lat=observatory[1]*u.deg, lon=observatory[2]*u.deg,height=observatory[3]*u.m)

#use location and time to get information on planets
moon = get_moon(t, loc)
sun = get_sun(t)
with solar_system_ephemeris.set('builtin'):
    merc = get_body('mercury', t, loc) 
with solar_system_ephemeris.set('builtin'):
    venus = get_body('venus', t, loc) 
with solar_system_ephemeris.set('builtin'):
    mars = get_body('mars', t, loc) 
with solar_system_ephemeris.set('builtin'):
    jup = get_body('jupiter', t, loc) 
with solar_system_ephemeris.set('builtin'):
    sat = get_body('saturn', t, loc) 
with solar_system_ephemeris.set('builtin'):
    nep = get_body('neptune', t, loc) 

warnings.filterwarnings("ignore")

#create grid for displaying output plots next to one another
grid = widgets.Grid(1, 2)

#compute altitude and azimuth for planets, use location info to create
#observer object and use to create skyplot over 24 hr period
with grid.output_to(0, 0):
  altair1 = FixedTarget(name='saturn', coord=sat)
  altair2 = FixedTarget(name='neptune', coord=nep)
  altair3 = FixedTarget(name='jupiter', coord=jup)
  altair4 = FixedTarget(name='moon', coord=moon)
  altair5 = FixedTarget(name='mars', coord=mars)
  altair6 = FixedTarget(name='venus', coord=venus)
  altair7 = FixedTarget(name='mercury', coord=merc)
  observe_time = t + np.arange(-12,13)*u.hour
  fig = plt.figure(figsize=(20, 10))
  ax = fig.add_subplot()
  obs_location = Observer(longitude=observatory[2]*u.deg, latitude=observatory[1]*u.deg,
                  elevation=observatory[3]*u.m, name=observatory[0], timezone=observatory[4])
  style1 = {'marker': '+','s': 100}
  style2 = {'marker': 'o','s': 100}
  style3 = {'marker': 'd','s': 100}
  style4 = {'marker': 'x','s': 100}
  style5 = {'marker': 's','s': 100}
  style6 = {'marker': '8','s': 100}
  style7 = {'marker': '*','s': 100}
  plot_sky(altair1, obs_location, observe_time,style_sheet=None, style_kwargs = style1)
  plot_sky(altair2, obs_location, observe_time,style_sheet=None, style_kwargs = style2)
  plot_sky(altair3, obs_location, observe_time,style_sheet=None, style_kwargs = style3)
  plot_sky(altair4, obs_location, observe_time,style_sheet=None, style_kwargs = style4)
  plot_sky(altair5, obs_location, observe_time,style_sheet=None, style_kwargs = style5)
  plot_sky(altair6, obs_location, observe_time,style_sheet=None, style_kwargs = style6)
  plot_sky(altair7, obs_location, observe_time,style_sheet=None, style_kwargs = style7)
  plt.title('Sky Plot for ' + observatory[0] + ' of the Planets on '+obs_date, fontsize=25)
  plt.legend(loc='center left', bbox_to_anchor=(1.25, 0.5),fontsize=20)
  plt.tight_layout()

#use same time, altitude, azimuth, and time objects above to plot altitude over
#24-hr period
with grid.output_to(0, 1):
  fig = plt.figure(figsize=(20, 10))
  ax = fig.add_subplot()
  style1 = {'linewidth': 3}
  plot_altitude(altair1, obs_location, t2,style_sheet=None, brightness_shading=True,style_kwargs = style1)
  plot_altitude(altair2, obs_location, t2,style_sheet=None, style_kwargs = style1)
  plot_altitude(altair3, obs_location, t2,style_sheet=None, style_kwargs = style1)
  plot_altitude(altair4, obs_location, t2,style_sheet=None, style_kwargs = style1)
  plot_altitude(altair5, obs_location, t2,style_sheet=None, style_kwargs = style1)
  plot_altitude(altair6, obs_location, t2,style_sheet=None, style_kwargs = style1)
  plot_altitude(altair7, obs_location, t2,style_sheet=None, brightness_shading=True,style_kwargs = style1)
  plt.title('Altitude Plot for ' + observatory[0] + ' of the Planets on '+obs_date, fontsize=25)
  plt.legend(loc='center left', bbox_to_anchor=(1, 0.5),fontsize=20)
  plt.show()

##Astronomers define twilight to end when the Sun is 18&deg; below the horizon, and corresponds to when the sky is no longer illuminated by the Sun. (After Sunset, atmospheric refraction allows the Sun to continue to illuminate the sky for some time.)

##<u>*Step 4*</u> Run the code below to see the Sunset, Sunrise, Moonrise and Moonset times (in local time) for the current date, along with information on when Twilight ends and begins and when the center of the night occurs. 



In [ ]:
#@title  { vertical-output: true, display-mode: "form" }

#create Pandas dataframe to display table of information
time_df = pd.DataFrame(new_list)
time_df = time_df.T
time_df.columns = ['Moonrise','Moonset','Sunrise','Sunset','Twilight Ends','Twilight Begins','Solar Midnight']
time_df.index = [str(obs_choice)]
time_df

---

##**1a.** (1/3 point) How long does twilight last at the start and end of the night in Hanover on April 1st?

### Please type your answer here

##**1b.** (1/3 point) Using the form in <u>*Step 2*</u> that sets the observation site (obs_choice), change your site to Mauna Kea, which is located on the Big Island in Hawaii. How long does twilight last in Mauna Kea on April 1st?

### Please type your answer here

##**1c.** (1/3 point) Why is the length of twilight different from Hanover? *Hint:* look at the latitude of the two sites, and think about the apparent motion of the Sun in the sky from each location.

### Please type your answer here

---

##Astronomers use the term **Airmass** to describe the length of the Earth's atmosphere which light travels through when observing a celestial source. The minimum pathlength occurs for an object at the zenith (directly overhead). The airmass at the zenith is set equal to 1 by definition (when the object is at the zenith z = 0, and sec(0&deg;)= 1). When the zenith angle z, which is the angle between the zenith and the celestial source, is less than 60&deg; then the airmass X can be approximated well via the secant of z, so X = sec(z). 

##Astronomers like to observe objects at fairly low values of the airmass (usually less than 2), as the earth's atmosphere absorbs and scatters light. Thus, an important part of preparing to observe involves selecting objects which are visible at low airmass from that observatory, at that time of year. 

##<u>*Step 5*</u> The star **Arcturus** is the third brightest star in the night sky and is located at Right Ascension (RA)= 14 15 39.7 and Declination (Dec)= +19 10 56. These coordinates for Arcturus have already been entered into the form below for you, so go ahead and run the block of code.


In [ ]:
#@title Arcturus RA and Dec
arcturus_RA = "14 15 39.7 " #@param {type:"string"}
arcturus_dec = "+19 10 56" #@param {type:"string"}
object_name = 'Arcturus'
ra_dec = arcturus_RA + ' ' + arcturus_dec

#use RA and Dec inputs to create SkyCoord object
arcturus = SkyCoord(ra_dec, unit=(u.hourangle, u.deg))
print("You successfully set the RA and dec for Arcturus")

##<u>*Step 6*</u> You will now use a few different tools to determine the best time to observe your objects. First you will look at a plot of airmass for Arcturus as a function of time during the selected night. It also prints out the *Meridian Transit time* (in UTC), which is the time at which an object crosses the Meridian. 

In [ ]:
#@title  { vertical-output: true, display-mode: "form" }

#use object info and location to plot airmas for Arcturus
transit_time = plot_object_airmass(object_name,obs_location,obs_date_time,obs_choice)
print('The time when Arcturus transits the Meridian'+ ' at '+obs_choice+ ' is ',str(transit_time[1]))

##Below you will calculate the *Hour Angle* of Arcturus at different times. 

##**Hour Angle (HA)** is defined to be HA = Sidereal Time - RA. As  a reminder, Sidereal Time is based on Earth's rate of rotation measured relative to the fixed stars, such that a Sidereal Day is the amount of time needed for the Earth to complete one rotation (slightly less than 24 hours).

##**2.** (1 point) Below is a list of Sidereal times. Use the RA (in hours, minutes, seconds) of Arcturus to calculate the HA for each of these Sidereal times. *Begin by running the block of code to display Arcturus' RA and the three times you will be calculating the HA for.*





In [ ]:
#@title  { vertical-output: true, display-mode: "form" }

#get sidereal time for Arcturus at solar midnight
solar_midnight = moon_sun_times[6]
object_coord = SkyCoord.from_name(object_name,frame='icrs')

#get RA of Arcturus
object_ra = object_coord.ra*u.deg
object_ra = float(np.array(object_ra))
#convert RA from degrees to hours,min,sec
object_ra_hms = deg2HMS(ra=object_ra)
object_ra_hms = Angle(object_ra_hms, unit=u.hourangle)
print('Arcturus RA is: ',object_ra_hms)

#get the time Arcturus rises in the sky and add 2 hours
altair1 = FixedTarget(name=object_name, coord=object_coord)
time1 = obs_location.target_rise_time(obs_date_time,altair1)
time1 = time1 + 2*u.hour
time1.format = 'iso'
time1_sd = time1.sidereal_time('apparent', longitude = observatory[2])
print('a). ',str(time1_sd))

#get the time Arcturus sets in the sky and subtract 4 hours
time2 = obs_location.target_set_time(obs_date_time,altair1)
time2 = time2 + -4*u.hour
time2.format = 'iso'
time2_sd = time2.sidereal_time('apparent', longitude = observatory[2])
print('b). ',str(time2_sd))

#get the sidereal time at Solar Midnight
time3 = Time(solar_midnight)
time3_sd = time3.sidereal_time('apparent', longitude = observatory[2])
time3_sd = str(time3_sd)
print('c). ',str(time3_sd))

### **a).** Please type your answer here

### **b).** Please type your answer here

### **c).** Please type your answer here

##**3.** (1 point) What can you infer about the relationship between the HA and the Meridian Transit time?

### Please type your answer here

##**4.** (1 point) In Hanover on the night of April 1, 2020, when (in UTC time) will the airmass of Arcturus be less than 2?


### Please type your answer here

---

## <u>*Step 7*</u> In my own research, I study stars and exoplanets, which we will investigate later in this course. Three of the exoplanets that I think are pretty awesome are: 51 Peg b (the first discovered exoplanet around a star like the Sun), K2-25b (which Dartmouth graduate student Keighley Rockcliffe is studying), and GJ 1214b (a planet around a nearby M dwarf). The little 'b' indicates the planet that orbits the star, e.g. GJ 1214 is the star that GJ 1214b orbits. Find out the RA and Dec of these objects by typing in their names to SIMBAD http://simbad.u-strasbg.fr/simbad/sim-fid, an online database used by astronomers. You can use the RA and Dec provided by the ICRS coordinates, which are the first two numbers listed to the right of where it says "ICRS coord. (ep=J2000) :". The first number is the RA followed by the Dec.

In [ ]:
#@title input the RA and dec for the exoplanets mentioned above
Peg_b_RA = "0" #@param {type:"string"}
Peg_b_dec = "0" #@param {type:"string"}
K2_25b_RA = "0" #@param {type:"string"}
K2_25b_dec = "0" #@param {type:"string"}
GJ_1214b_RA = "0" #@param {type:"string"}
GJ_1214b_dec = "0" #@param {type:"string"}

#use RA and Dec of the exoplants to create SkyCoord obejcts for each 
Peg_ra_dec = Peg_b_RA + ' ' + Peg_b_dec
Peg_b = SkyCoord(Peg_ra_dec, unit=(u.hourangle, u.deg))

K2_25b_ra_dec = K2_25b_RA + ' ' + K2_25b_dec
K2_25b = SkyCoord(K2_25b_ra_dec, unit=(u.hourangle, u.deg))

GJ_1214b_ra_dec = GJ_1214b_RA + ' ' + GJ_1214b_dec
GJ_1214b = SkyCoord(GJ_1214b_ra_dec, unit=(u.hourangle, u.deg))

print('You have successfully input the RA and Decs!')

##<u>*Step 8*</u> I would like to use the 2.4m telescope at MDM Observatory on Kitt Peak to observe all three of these stars/planets in December and at an airmass less than 1.5. I asked for time on the telescope in December 2020 to observe these stars. Because these stars are bright, I asked for "bright time" which is when the moon is close to full (fainter targets require "dark time", which is when it's a new moon). 


In [ ]:
#@title Change the day in December and run the block of code to see the moon phase/illumination for different days at solar midnight. Use this to find the best "bright time"
dec_obs_date =  "2020-12-01"#@param {type:"date"}
dec_obs_date_time = dec_obs_date + ' 00:00:00'

#create Time object based on input date/time
time = Time(dec_obs_date_time)
dec_obs_choice = "Kitt Peak Observatory, Arizona"
dec_obs_lat =31.951841
dec_obs_long =-111.615948
dec_obs_elev = 1923
dec_obs_tz = "US/Mountain"
dec_obs_utc = -7
#store observatory variables in list
dec_observatory = [dec_obs_choice,dec_obs_lat,dec_obs_long,dec_obs_elev,dec_obs_tz,dec_obs_utc]
#create Observer location object from observatory information
dec_obs_location = Observer(longitude=dec_observatory[2]*u.deg, latitude=dec_observatory[1]*u.deg,
                  elevation=dec_observatory[3]*u.m, name=dec_observatory[0], timezone=dec_observatory[4])

#get solar midnight based on time input and compute moon phase and illumination
dec_solar_midnight = dec_obs_location.midnight(time)
illumination = dec_obs_location.moon_illumination(dec_solar_midnight) 
phase = dec_obs_location.moon_phase(dec_solar_midnight) 
print('Moon Illumination as a fraction of lunar surface illumination (1 = fully illuminated)')
print('Moon Phase in radians where pi = new moon and zero = full moon')
print('Moon Illumination: ',illumination,',','Moon Phase: ',phase)

#from names of the three objects create SkyCoord, FixedTarget, and EarthLocation objects
Peg_b = SkyCoord.from_name('51 Peg b')  
target1 = FixedTarget(name='Peg_b', coord=Peg_b,frame ='icrs')
K2_25b = SkyCoord.from_name('K2-25b')  
target2 = FixedTarget(name='K2_25b', coord=K2_25b,frame ='icrs')
GJ_1214b = SkyCoord.from_name('GJ 1214b')
target3 = FixedTarget(name='GJ_1214b', coord=GJ_1214b,frame ='icrs')
dec_loc = EarthLocation(lat=dec_obs_lat*u.deg, lon=dec_obs_long*u.deg,height=dec_obs_elev*u.m)
moon = get_moon(dec_solar_midnight, dec_loc)
target4 = FixedTarget(name='Moon', coord=moon,frame ='icrs')

#plot the airmass of all the objects plus the moon over a 24-hr period
fig, ax = plt.subplots(figsize=(20,10))
plt.yticks(fontsize=20)
plt.xticks(fontsize=20)
plt.xlabel('Time',fontsize=25)
plt.ylabel('Altitude',fontsize=25)
style1 = {'linewidth': 5}
plot_airmass(target1, dec_obs_location, dec_solar_midnight, style_kwargs=style1)
plot_airmass(target2, dec_obs_location, dec_solar_midnight, style_kwargs=style1)
plot_airmass(target4, dec_obs_location, dec_solar_midnight, style_kwargs=style1)
plot_airmass(target3, dec_obs_location, dec_solar_midnight, brightness_shading=True,altitude_yaxis=True,style_kwargs=style1)
ax.legend(shadow=True, loc=2)
plt.yticks(fontsize=20)
plt.ylabel('Altitude [degrees]',fontsize=25)
plt.show()


##**5.** (1 point) Were these observations possible on a single night from MDM during December 2020? If so, which date should I observe? If not, when might I have asked for additional telescope time? Explain your answer, which can include output from Airmass plot.

Please type your answer here

---

###**Downloading Instructions:** When your group has completed this exercise, download a .pdf of this notebook and upload it to Canvas. You can do this by going to File>Print>Destination>Save to PDF. 

> *Note:* you might have to play with the document scale (under More Settings) to ensure plots are not cut off. If you have difficulty producing the .pdf without errors try using a different web browser.